In [ ]:
# =========================
# IMPORT LIBARY ===========
# =========================

import pandas as pd # mengimpor libary pandas untuk manipulasi data
import numpy as np # mengimpor libary numpy untuk komputasi numerik
import matplotlib.pyplot as plt # mengimpor matplotlib untuk visualisasi data
import seaborn as sns # mengimpor libary seaborn untuk visualisasi data

data_path = 'datasets/diabetes_data_upload.csv'

df = pd.read_csv(data_path)

df.head()

In [ ]:
# =====================
# DATA PREPROCESSING ==
# =====================

df.isnull().sum()
df.duplicated().sum()

df = df.drop_duplicates().reset_index(drop=True)
df = df.fillna(df.mode().iloc[0])


def encode_column(col):
    unique_vals = col.unique()
    mapping = {val: idx for idx, val in enumerate(unique_vals)}
    return col.map(mapping)


for col in df.columns:
    if df[col].dtype == "object":
        df[col] = encode_column(df[col])